In [1]:
import pandas as pd
import numpy as np
import pygmt
import sys
import os, glob
from os import listdir
from os.path import isfile, join
from obspy import read
import warnings
from obspy.core import UTCDateTime
# ----------------- Turn off warning for all programs -------------------------
warnings.filterwarnings('ignore')
# -----------------------------------------------------------------------------
input_path="input/sac_202404022358_200_TSMIP"
# output directory
out_dir="output/cwb_202404022358_200_TSMIP"
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
# 
# files in the input directory
files = [f for f in glob.glob(input_path+"/*.sac")]
# find all stations names
stas = []; chans = [];
for i,file in enumerate(files):
    sta = file.split("/")[2].split(".")[0]
    chan = file.split("/")[2].split(".")[1]
    stas = np.append(stas,sta)
    chans = np.append(chans,chan)
# stas = np.unique(stas)
# stas = ["A001"]; # for checking purpose
chans = np.unique(chans)
# Process loop for all stations
for i,sta in enumerate(stas):
    c1=[];c2=[];c3=[]; st=[];# check point
    npts = 0; npts0 = 0; npts1 = 0; npts2= 0;
    # check 3 component of stations
    c1=[x for x in files if sta+'.'+chans[0] in x]
    c2=[x for x in files if sta+'.'+chans[1] in x]
    c3=[x for x in files if sta+'.'+chans[2] in x]
    if ((np.size(c1)!=0) and (np.size(c1)!=0) and (np.size(c1)!=0)):
        st = read(c1[0]) # E
        st += read(c2[0]) # N
        st += read(c3[0]) # Z
    # Check 3 components consistency
    if (st[0].stats.station != st[1].stats.station != st[2].stats.station): print("inconsistent station name!",st[0].stats.station); sys.exit(0)
    if (st[0].stats.starttime != st[1].stats.starttime != st[2].stats.starttime): print("inconsistent starttimes!",st[0].stats.starttime,st[1].stats.starttime,st[2].stats.starttime); sys.exit(0)
    # if (st[0].stats.endtime != st[1].stats.endtime != st[2].stats.endtime): print("inconsistent endtimes!",st[0].stats.endtime,st[1].stats.endtime,st[2].stats.endtime); sys.exit(0)
    if (st[0].stats.sampling_rate != st[1].stats.sampling_rate != st[2].stats.sampling_rate): print("inconsistent sampling_rate!",st[0].stats.sampling_rate,st[1].stats.sampling_rate,st[2].stats.sampling_rate); sys.exit(0)
    # if (st[0].stats.npts != st[1].stats.npts != st[2].stats.npts): 
    #     print("inconsistent data length!",st[0].stats.npts,st[1].stats.npts,st[2].stats.npts)
    npts0 = st[0].stats.npts
    npts1 = st[1].stats.npts
    npts2 = st[2].stats.npts
        # find minimum value of 3 values
    npts = min([npts0,npts1,npts2])
        
    if (st[0].stats.sac.stla != st[1].stats.sac.stla != st[2].stats.sac.stla): print("inconsistent station latitude!",st[0].stats.sac.stla,st[1].stats.sac.stla,st[2].stats.sac.stla); sys.exit(0)
    if (st[0].stats.sac.stlo != st[1].stats.sac.stlo != st[2].stats.sac.stlo): print("inconsistent station longitude!",st[0].stats.sac.stlo,st[1].stats.sac.stlo,st[2].stats.sac.stlo); sys.exit(0)
    if (st[0].stats.sac.stel != st[1].stats.sac.stel != st[2].stats.sac.stel): print("inconsistent station longitude!",st[0].stats.sac.stel,st[1].stats.sac.stel,st[2].stats.sac.stel); sys.exit(0)
    # end of check the header
    #  ----- Start calculate the value for the header of CWB file ----
    stacode=st[0].stats.station.strip() # 1st line
    stalat=st[0].stats.sac.stla # 2nd line
    stalon=st[0].stats.sac.stlo # 3rd line
    evtlat=st[0].stats.sac.evla # 4th line
    evtlon=st[0].stats.sac.evlo # 5th line
    evtml=st[0].stats.sac.mag # 6th line
    evtmw=st[0].stats.sac.user9 # 7th line
    evtdep=st[0].stats.sac.evdp # 8th line
    instyp="N.A." # 9th line
    # 10th line forming by utcdatetime
    starttime="%4.4d%2.2d%2.2d%2.2d%2.2d%2.2d"%(st[0].stats.starttime.year,st[0].stats.starttime.month,st[0].stats.starttime.day,st[0].stats.starttime.hour,st[0].stats.starttime.minute,st[0].stats.starttime.second)
    samprate=st[0].stats.sampling_rate # 11th line
    # 12th check the unit
    unit="cm/ss"
    #  ----- file name make ----
    evtotime=UTCDateTime(year=st[0].stats.sac.nzyear, julday=st[0].stats.sac.nzjday, hour=st[0].stats.sac.nzhour,minute=st[0].stats.sac.nzmin,second=st[0].stats.sac.nzsec)
    eventotime="%4.4d%2.2d%2.2d%2.2d%2.2d%2.2d"%(evtotime.year,evtotime.month,evtotime.day,evtotime.hour,evtotime.minute,evtotime.second)
    outfile = st[0].stats.station+"_"+str(eventotime)+".cwb"
    # Now write header of file
    print("Writing output: ",outfile)
    with open(out_dir+"/"+outfile,'w+') as f:
        f.write('%StationCode: {}\n'.format(stacode))
        f.write('%StationLat: {:10.5f}\n'.format(stalat))
        f.write('%StationLon: {:10.5f}\n'.format(stalon))
        f.write('%EventLat: {:5.2f}\n'.format(evtlat))
        f.write('%EventLon: {:5.2f}\n'.format(evtlon))
        f.write('%EventML: {:4.2f}\n'.format(evtml))
        f.write('%EventMW: {:4.2f}\n'.format(evtmw))
        f.write('%EventDep: {:5.2f}\n'.format(evtdep))
        f.write('%InstrumentKind: {}\n'.format(instyp))
        f.write('%StartTime: {}\n'.format(starttime))
        f.write('%SampleRate(Hz): {:d}\n'.format(int(samprate)))
        f.write('%AmplitudeUnit: {}\n'.format(unit))
        f.write('%AmplitudeMAX. U:{:7.3f}~{:7.3f}\n'.format(st[2].stats.sac.depmax, st[2].stats.sac.depmin))
        f.write('%AmplitudeMAX. N:{:7.3f}~{:7.3f}\n'.format(st[1].stats.sac.depmax, st[1].stats.sac.depmin))
        f.write('%AmplitudeMAX. E:{:7.3f}~{:7.3f}\n'.format(st[0].stats.sac.depmax, st[0].stats.sac.depmin))
        f.write('%DataSequence: Time U(+); N(+), E(+)\n')
        f.write('%Data: 4F10.3\n')
        for i in range(npts):
            f.write('%10.3f%10.3f%10.3f%10.3f\n'%(st[0].times()[i],st[2].data[i],st[1].data[i],st[0].data[i]))
    f.close()
print("Process finish!!!")

Writing output:  D117_20240402235800.cwb
Writing output:  D117_20240402235800.cwb
Writing output:  D117_20240402235800.cwb
Process finish!!!


In [ ]:
'''
Check max npts
'''
import pandas as pd
import numpy as np
import pygmt
import sys
import os, glob
from os import listdir
from os.path import isfile, join
from obspy import read
import warnings
from obspy.core import UTCDateTime
# ----------------- Turn off warning for all programs -------------------------
warnings.filterwarnings('ignore')
# -----------------------------------------------------------------------------
input_path="input/sac_202404022358_200_TSMIP"
files = [f for f in glob.glob(input_path+"/*.sac")]
# find all stations names
npts=[];
for i,file in enumerate(files):
    st=read(file)
    npts=np.append(npts,st[0].stats.npts)
np.max(npts)